In [27]:
from dotenv import load_dotenv
import os
import requests

# Load the environment variables from the .env file
load_dotenv()

connection_string = os.getenv('SQL_CONNECTION')

## Test database exists

In [32]:
import sqlitecloud

# Open the connection to SQLite Cloud
conn = sqlitecloud.connect(connection_string)

# You can autoselect the database during the connect call
# by adding the database name as path of the SQLite Cloud
# connection string, eg:
# conn = sqlitecloud.connect("sqlitecloud://myhost.sqlite.cloud:8860/mydatabase?apikey=myapikey")
db_name = "chinook.sqlite"
conn.execute(f"USE DATABASE {db_name}")

cursor = conn.execute("SELECT * FROM waves")
result = cursor.fetchall()

print(result)

conn.close()

[('2024-08-15 00:00:00', 45.0, 1.5, 5.5), ('2024-08-15 00:30:00', 50.0, 2.0, 6.0), ('2024-08-15 01:00:00', 30.0, 1.2, 5.2)]


## Test push/overwrite data

In [18]:
import pandas as pd

# Example: Assume `predictions_df` is the DataFrame output from your model
predictions_df = pd.DataFrame({
    'wave_direction': [45.0, 50.0, 30.0],
    'wave_height': [1.5, 2.0, 1.2],
    'wave_period': [5.5, 6.0, 5.2]
}, index=pd.to_datetime(['2024-08-15 00:00:00', '2024-08-15 00:30:00', '2024-08-15 01:00:00']))
predictions_df.index.name = 'datetime'
predictions_df

,wave_direction,wave_height,wave_period
datetime,,,
2024-08-15 00:00:00,45.0,1.5,5.5
2024-08-15 00:30:00,50.0,2.0,6.0
2024-08-15 01:00:00,30.0,1.2,5.2


In [30]:
import sqlitecloud
import pandas as pd

# Example DataFrame output from your model
predictions_df = pd.DataFrame({
    'wave_direction': [45.0, 50.0, 30.0],
    'wave_height': [1.5, 2.0, 1.2],
    'wave_period': [5.5, 6.0, 5.2]
}, index=pd.to_datetime(['2024-08-15 00:00:00', '2024-08-15 00:30:00', '2024-08-15 01:00:00']))
predictions_df.index.name = 'datetime'
# Convert datetime index to ISO 8601 string format
predictions_df.index = predictions_df.index.strftime('%Y-%m-%d %H:%M:%S')

# Connect to the SQLiteCloud database
conn = sqlitecloud.connect(connection_string)
db_name = "chinook.sqlite"
conn.execute(f"USE DATABASE {db_name}")

def upsert_row(conn, datetime, wave_direction, wave_height, wave_period):
    upsert_sql = f'''
    INSERT INTO waves (datetime, wave_direction, wave_height, wave_period)
    VALUES (?, ?, ?, ?)
    ON CONFLICT(datetime) DO UPDATE SET
        wave_direction=excluded.wave_direction,
        wave_height=excluded.wave_height,
        wave_period=excluded.wave_period;
    '''
    conn.execute(upsert_sql, (datetime, wave_direction, wave_height, wave_period))

try:
    for index, row in predictions_df.iterrows():
        upsert_row(conn, index, row['wave_direction'], row['wave_height'], row['wave_period'])
    
    print("Data appended and updated successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()


Data appended and updated successfully.
